# Kafka

## Kafka相关概念

Kakfa定位为一个分布式流式处理平台，高吞吐、可持久化、可水平扩展、支持流式数据处理。kafka可以被用作：

* 消息系统，实现系统解耦、冗余存储、流量削峰，kafka还提供了大多数消息系统难以实现的**消息顺序性保障**和**消息回溯**功能
* 存储系统，kafka把消息持久化到磁盘
* 流式处理平台

### kafka技术架构

![](https://bj.bcebos.com/ipic/Kafka体系架构.jpg)

* Producer生产者，发送消息的一方，负责把消息发送到kafka中
* Consumer消费者，接收消息的一方，消费者连接到kafka上接收消息，并处理相应的业务逻辑
* Broker服务代理节点，Broker可以看做一个Kakfa服务节点或者实例。
* ZooKeeper，管理集群的元数据

## kafka事务

### 消息传输保障

消息传输保障有3个层级：

* `at most once`: 至多一次，消息可能会丢失，但绝对不会重复传输
* `at least once`: 最少一次，消息绝对不会丢失，但可能会重复传输
* `exactly once`: 恰好一次，消息肯定会被传输，且只传输1次

kafka从`0.11.0.0`引入了幂等和事务两个特性，以此实现EOS(Exactly Once Sematics)。

### 幂等

幂等是对接口的多次调用和一次调用产生的结果是一致的。kafka生产者再进行重试的时候可能会重复写入消息，而使用kafka的幂等性功能后可以避免这种情况。

为了实现生产者的幂等性，kafka引入了producer id（简称PID）和sequence number（序列号）。

每个新的生产者实例在初始化的时候都会分配一个PID，这个PID对用户完全是不可见的。

broker端会在内存中为每堆<PID, Partition>维护一个序列号SN_old，新消息的序列号SN_new

* 新消息序列号SN_new比broker维护的序列号SN_old大于1，broker接收该消息
* SN_new小于SN_old+1，说明消息重复写入，该消息被丢弃
* SN_new大于SN_old+1，说明中间有消息未写入，出现了乱序，broker会抛出`OutOfOrderSequenceException`


## FAQ